# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [2]:
df['id'].describe()

count      8188.000000
mean     564699.749878
std      219085.845672
min      182988.000000
25%      373637.500000
50%      566840.500000
75%      758078.000000
max      937217.000000
Name: id, dtype: float64

In [3]:
df['id'].value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [4]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [5]:
df.isna().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [6]:
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [12]:
dupe_id = df.loc[df.duplicated(subset=['id'], keep=False)]
dupe_id

,timestamp,id,group,action
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
11,2016-09-24 21:05:15.348935,601714,experiment,view
12,2016-09-24 21:06:27.553057,601714,experiment,click
14,2016-09-24 21:29:19.766467,487634,experiment,view
...,...,...,...,...
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8179,2017-01-18 08:53:50.910310,615849,experiment,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8181,2017-01-18 09:07:37.661143,795585,control,view


In [19]:
dupe_control = df.loc[
    (df['id'].isin(dupe_id['id']))
    & (df['group'] == 'control')
]

dupe_experiment = df.loc[
    (df['id'].isin(dupe_id['id']))
    & (df['group'] == 'experiment')
]

both_groups = df.loc[
    (df['id'].isin(dupe_control['id']))
    & (df['id'].isin(dupe_experiment['id']))
]
both_groups

,timestamp,id,group,action


In [23]:
dupe_view = df.loc[
    (df['id'].isin(dupe_id['id']))
    & (df['action'] == 'view')
]

dupe_click = df.loc[
    (df['id'].isin(dupe_id['id']))
    & (df['action'] == 'click')
]

click_noView = df.loc[
    (df['id'].isin(dupe_click['id']))
    & (~df['id'].isin(dupe_view['id']))
]

click_noView

,timestamp,id,group,action


In [24]:
clickAndView = df.loc[
    (df['id'].isin(dupe_click['id']))
    & (df['id'].isin(dupe_view['id']))
]
clickAndView

,timestamp,id,group,action
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
11,2016-09-24 21:05:15.348935,601714,experiment,view
12,2016-09-24 21:06:27.553057,601714,experiment,click
14,2016-09-24 21:29:19.766467,487634,experiment,view
...,...,...,...,...
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8179,2017-01-18 08:53:50.910310,615849,experiment,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8181,2017-01-18 09:07:37.661143,795585,control,view


In [29]:
len(clickAndView['id'].unique())

1860

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [52]:
#Your code here
experiment = df.loc[df['group'] == 'experiment']
control = df.loc[df['group'] == 'control']

count_exp = experiment['action'].value_counts()
count_con = control['action'].value_counts()
count_exp['view'], count_exp['click'], count_con['view'], count_con['click']

(2996, 928, 3332, 932)

In [53]:
exp_ratio = count_exp['click'] / (count_exp['view'])
con_ratio = count_con['click'] / (count_con['view'])
exp_ratio, con_ratio

(0.3097463284379172, 0.2797118847539016)

In [55]:
import scipy.stats as stats


control_clicked = control.loc[control['action'] == 'click']
con_click = control.drop(['timestamp'], axis=1)
con_click.loc[con_click['id'].isin(control_clicked['id']), 'clicked'] = 1
con_click['clicked'].fillna(0, inplace=True)
con_click.drop_duplicates(subset=['id'], inplace=True)
con_click.drop(['action', 'group'], axis=1, inplace=True)

experiment_clicked = experiment.loc[experiment['action'] == 'click']
exp_click = experiment.drop(['timestamp'], axis=1)
exp_click.loc[exp_click['id'].isin(experiment_clicked['id']), 'clicked'] = 1
exp_click['clicked'].fillna(0, inplace=True)
exp_click.drop_duplicates(subset=['id'], inplace=True)
exp_click.drop(['action', 'group'], axis=1, inplace=True)


In [56]:
exp_click

,id,clicked
0,804196,0.0
1,434745,0.0
2,507599,0.0
4,536734,0.0
5,681598,0.0
...,...,...
8176,540466,0.0
8179,615849,1.0
8183,192060,0.0
8184,755912,0.0


In [57]:
con_click

,id,clicked
3,671993,0.0
9,560027,0.0
23,281985,0.0
24,407864,0.0
25,342984,0.0
...,...,...
8177,839581,0.0
8178,616692,0.0
8181,795585,1.0
8186,505451,0.0


In [62]:
result = stats.ttest_ind(con_click['clicked'], exp_click['clicked'], equal_var=False)
result.pvalue / 2, result.statistic

(0.004466402814337102, -2.615440020788211)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [60]:
#Your code here
expect_exp_click = con_ratio * count_exp['view']
expect_exp_click

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [70]:
#Your code here
var = count_exp['view'] * con_ratio * (1-con_ratio)
import numpy as np
z = (count_exp['click'] - expect_exp_click) / np.sqrt(var)
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [71]:
#Your code here
stats.norm.sf(z)

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Both p-values are comfortably below 0.05, so we can reject that the experimental homepage performed equal to or worse than the control homepage

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.